## 使用网络爬虫抓取明星图片

In [108]:
import requests
from urllib import parse
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
img_url = 'http://image.baidu.com/search/acjson?tn=resultjson_com&ipn=rj&is=&fp=result&word={0}&step_word={0}&pn={1}'

def getIdolPicture(keyword, dest_dir, batch):
    if not os.path.exists(dest_dir):
        os.mkdir(dest_dir)
    for i in range(batch):
        res = requests.get(img_url.format(parse.quote(keyword), i * 30))
        res.encoding = 'utf-8'
        for ele in res.json()['data']:
            url = ele.get('thumbURL')
            if url:
                with open(dest_dir + url.split('/')[-1], 'wb') as f:
                    res2 = requests.get(url, headers = headers)
                    f.write(res2.content)

In [ ]:
getIdolPicture('范冰冰', 'idol1/',10)

## 使用OpenCV 撷取人脸

In [3]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
face_cascade = cv.CascadeClassifier('/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml')
from PIL import Image
def extractFace(filename, src_dir, dest_dir):
    img = cv.imread(src_dir + filename)
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    im = Image.open(src_dir + filename)
    for (x,y,w,h) in faces:
        box = (x, y, x+w, y+h)
        crpim = im.crop(box).resize((64,64))
        crpim.save(dest_dir + filename)

In [5]:
import os
os.mkdir('idol1_face')
for f in os.listdir('idol1/'):
    try:
        extractFace(f, 'idol1/', 'idol1_face/')
    except:
        print(f)

.DS_Store


## 利用卷积神经网络辨识明星图片

### 建构卷积神经网络

In [7]:
from keras.models import Sequential 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D 
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()
# Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64,
3), activation = 'relu'))
# Max Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Convolution
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
# Max Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Flattening
classifier.add(Flatten())

# Fully Connected
classifier.add(Dense(units = 128, activation = 'relu')) 
classifier.add(Dense(units = 3, activation = 'softmax'))

classifier.compile(optimizer = 'adam', 
                        loss ='categorical_crossentropy', 
                     metrics = ['accuracy'])

### 图像增广技术

In [8]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,     #x坐标保持不变，而对应的y坐标按比例发生平移
                                   zoom_range = 0.2,      #可以让图片在长或宽的方向进行放大
                                   horizontal_flip = True #水平翻转操作
                                  )

In [9]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### 建立训练与测试数据集

In [10]:
training_set = train_datagen.flow_from_directory(
    'trainset/', target_size = (64, 64),
     batch_size = 30,
     class_mode = 'categorical')

Found 413 images belonging to 3 classes.


In [11]:
test_set = test_datagen.flow_from_directory(
    'testset/', target_size = (64, 64),
    batch_size = 30, 
    class_mode = 'categorical')

Found 106 images belonging to 3 classes.


### 训练神经网路

In [12]:
history = classifier.fit_generator(training_set,
                         epochs = 100,
                         verbose = 1, 
                         validation_data = test_set)

Epoch 1/100
14/14 [==============================] - 2s 152ms/step - loss: 1.2127 - acc: 0.3030 - val_loss: 1.0971 - val_acc: 0.4245
Epoch 2/100
14/14 [==============================] - 1s 105ms/step - loss: 1.0901 - acc: 0.4239 - val_loss: 1.1323 - val_acc: 0.3396
Epoch 3/100
14/14 [==============================] - 1s 101ms/step - loss: 1.0791 - acc: 0.4259 - val_loss: 1.0750 - val_acc: 0.3396
Epoch 4/100
14/14 [==============================] - 1s 100ms/step - loss: 1.0322 - acc: 0.4202 - val_loss: 1.0209 - val_acc: 0.3774
Epoch 5/100
14/14 [==============================] - 1s 101ms/step - loss: 0.9745 - acc: 0.5189 - val_loss: 0.9681 - val_acc: 0.4717
Epoch 6/100
14/14 [==============================] - 1s 104ms/step - loss: 0.9489 - acc: 0.5492 - val_loss: 0.9928 - val_acc: 0.4245
Epoch 7/100
14/14 [==============================] - 1s 106ms/step - loss: 0.8914 - acc: 0.5979 - val_loss: 0.9520 - val_acc: 0.4434
Epoch 8/100
14/14 [==============================] - 2s 107ms/step - 

14/14 [==============================] - 1s 104ms/step - loss: 0.2144 - acc: 0.9128 - val_loss: 0.7816 - val_acc: 0.7642
Epoch 63/100
14/14 [==============================] - 1s 104ms/step - loss: 0.2742 - acc: 0.8941 - val_loss: 0.6418 - val_acc: 0.8019
Epoch 64/100
14/14 [==============================] - 1s 103ms/step - loss: 0.1703 - acc: 0.9241 - val_loss: 0.6326 - val_acc: 0.7925
Epoch 65/100
14/14 [==============================] - 1s 102ms/step - loss: 0.2003 - acc: 0.9319 - val_loss: 0.6366 - val_acc: 0.8019
Epoch 66/100
14/14 [==============================] - 1s 101ms/step - loss: 0.1568 - acc: 0.9486 - val_loss: 0.5784 - val_acc: 0.8208
Epoch 67/100
14/14 [==============================] - 1s 100ms/step - loss: 0.1665 - acc: 0.9462 - val_loss: 0.6289 - val_acc: 0.8302
Epoch 68/100
14/14 [==============================] - 1s 100ms/step - loss: 0.1900 - acc: 0.9241 - val_loss: 0.5779 - val_acc: 0.8585
Epoch 69/100
14/14 [==============================] - 1s 102ms/step - loss:

### 预测单张图片

In [195]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
face_cascade = cv.CascadeClassifier('/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml')
from PIL import Image

img = cv.imread('i1.jpg')
faces = face_cascade.detectMultiScale(img, 1.3, 5)
im = Image.open('i1.jpg')
x,y,w,h = faces[0]
box = (x, y, x+w, y+h)
crpim = im.crop(box).resize((64,64))
crpim.save('i1_d.jpg')

In [23]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('prediction_dataset/i2_d.jpg', target_size= (64,64))
test_image = image.img_to_array(test_image)

type(test_image)

numpy.ndarray

In [24]:
test_image.shape

(64, 64, 3)

In [25]:
test_image = np.expand_dims(test_image, axis = 0)

In [26]:
test_image.shape

(1, 64, 64, 3)

In [27]:
classifier.predict_classes(test_image)

array([1])